In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}

In [3]:
# 만들어뒀던 download 함수

def download(method="get", url="", headers={}, params={}, retries=3):
    try:
        resp = requests.request(method, 
                                url, 
                                headers=headers,
                                params=params if method=="get" else {},
                                data = params if method=="post" else {}
                               )
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print("Retries:",retries)
            time.sleep(random.randint(1,5))
            return download(method, url, headers, params, retries-1)
            
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            print(resp.url)

    return resp


# 구글에서 이미지 검색 결과를 scrap 해보자.

In [341]:
url = "https://www.google.com/search"
params = {
    "q":"성소",
    "source":"lnms",
    "tbm":"isch"
}
resp = download("get", url, headers=headers, params=params)

In [342]:
# 오늘은 parser를 바꿔보겠습니다.
# lxml로 parsing이 잘 안되면 html.parser가 좋습니다.
# 장점은 개떡같이 만든 html 코드도 잘 parsing한다.
# 단점은 느리다.

dom = BeautifulSoup(resp.text, "html.parser")

In [343]:
# 구글 이미지 검색 결과에서 이미지들은 다음과 같은 클래스를 갖고 있고, 해당 태그는 총 100개가 있다.

print(len(dom.select(".rg_ic.rg_i")))

100


In [344]:
# src 속성을 갖고 있는 것들만 걸러내면 이미지는 총 20개이다.

imgs = [_["src"] for _ in dom.select(".rg_ic.rg_i")
        if _.has_attr("src")]

print(len(imgs))

20


In [345]:
# base64 인코딩되어 있다.
# image를 link로 하는 것보다 base64가 더 안전하다.
# link로 이미지를 걸면 엑스박스가 뜰 위험이 있다.

imgs[-1]

'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='

In [31]:
import base64

temp = imgs[0].split(",")
ext = temp[0].split(";")[0].split("/")[-1]
img = base64.b64decode(temp[1].encode("ascii"))

In [32]:
with open("1."+ext, "wb") as fp:
    fp.write(img)

> base64로 이미지 저장하는 방법이 잘 안된다...
>
> 다른 방법을 찾아보자.

----

> 다시할게요,,,
>
> 이미지 소스가 json으로 되어있는 것이 문제였다.
>

In [346]:
# 이미지 소스정보가 담긴 json은 다음 태그에서 얻을 수 있다.

images = [_.text for _ in dom.select(".rg_meta.notranslate")]

In [347]:
# json 정보의 현재 type이 str이다.

print(type(images[0]))
images[0]

<class 'str'>


'{"cb":12,"cl":15,"cr":21,"id":"Xm3IhVOeFnfSAM:","isu":"news.chosun.com","itg":0,"ity":"jpg","oh":782,"ou":"http://image.chosun.com/sitedata/image/201809/03/2018090301334_0.jpg","ow":540,"pt":"SC이슈]\\"컴백 NO, 예능은 OK\\"\\u2026우주소녀 성소, 中스케줄 논란 ...","rh":"news.chosun.com","rid":"eNkCp7MbYFuoPM","rmt":0,"rt":0,"ru":"http://news.chosun.com/site/data/html_dir/2018/09/03/2018090301347.html","sc":1,"st":"조선일보","th":270,"tu":"https://encrypted-tbn0.gstatic.com/images?q\\u003dtbn:ANd9GcRWPUERnmXHTdJA81s9gdcNo5tk6X9XEshi_O7ukJdoLWZcHMCtxw","tw":186}'

In [348]:
# str로 되어있는 json을 python dictionary로 바꿔주자.

import json

img = json.loads(images[0])

type(img)

dict

In [349]:
# dictionary의 key 호출 방식을 사용할 수 있다.
# ou라는 key에 image 주소가 들어있다.

img["ou"]

'http://image.chosun.com/sitedata/image/201809/03/2018090301334_0.jpg'

In [ ]:
# 확장자가 제멋대로여도 일괄적으로 맞춰주기 위한 딕셔너리를 만듦.
ext = {
    "JPEG":"jpg",
    "JPG":"jpg",
    "PNG":"png",
    "GIF":"gif",
    "jpeg":"jpg",
    "jpg":"jpg",
    "png":"png",
    "gif":"gif"
}

# 선생님이 만드신 코드.
for _ in images[:10]:
    img = json.loads(_)
    resp = download("get", img["ou"], headers=headers)
    
    if resp.headers["Content-Type"].split("/")[0] == "image":
        imgTitle = resp.url.split("/")[-1].split(".")[0]
        imgExt = ext[resp.headers["Content-Type"].split("/")[-1]]
        with open(imgTitle+"."+imgExt, "wb") as fp:
            fp.write(resp.content)

---

In [104]:
# imgTitle에서 에러가 나서 tilte parsing부분을 수정함.

for _ in images[:10]:
    img = json.loads(_)
    resp = download("get", img["ou"], headers=headers)
    
    if resp.headers["Content-Type"].split("/")[0] == "image":
        if len(resp.url.split("/")[-1].split(".")[-1]) < 5:
            imgTitle = requests.utils.unquote(resp.url.split("/")[-1].split(".")[0])
        else:
            imgTitle = img["pt"] 
        imgExt = ext[resp.headers["Content-Type"].split("/")[-1]]
        with open(imgTitle+"."+imgExt, "wb") as fp:
            fp.write(resp.content)

> 아래는 위 코드를 부연 설명하기 위한 노트입니다.

In [105]:
# imgTitle이 퍼센트인코딩이 되어있으므로, 유니코드로 unquote하는 방식으로 수정함.

print(resp.url.split("/")[-1].split(".")[0])
print(requests.utils.unquote(resp.url.split("/")[-1].split(".")[0]))

170115_%EA%B0%95%EB%82%A8%ED%8C%AC%EC%8B%B8%EC%9D%B8%ED%9A%8C_%EC%9A%B0%EC%A3%BC%EC%86%8C%EB%85%80_%EC%9D%80%EC%84%9C_%EC%84%B1%EC%86%8C_%EC%97%AC%EB%A6%84_%281%29
170115_강남팬싸인회_우주소녀_은서_성소_여름_(1)


In [361]:
# url이 파일명을 포함하고 있는 경우에는 맨 마지막이 .jpg 등의 확장자로 끝난다

img = json.loads(images[0])
print(img["ou"])
resp.url.split("/")[-1].split(".")[-1]

http://image.chosun.com/sitedata/image/201809/03/2018090301334_0.jpg


'jpg'

In [112]:
# url 주소가 파일명을 포함하고 있지 않은 경우가 있으면 에러가 난다.

img = json.loads(images[8])
img["ou"]

'https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile2.uf.tistory.com%2Fimage%2F998C50505AEB18F33303CE'

In [88]:
# url이 파일명을 포함하고 있지 않은 경우에는 끝이 5글자를 넘어간다.
# 보통 이미지 파일의 확장자가 5글자를 넘어가는 경우는 거의 없으므로, 이를 if문으로 만들어 예외처리하였다.

len(resp.url.split("/")[-1].split(".")[-1]) < 5

36

In [362]:
# if로 예외처리했을떄는 "pt"를 파일의 제목으로 하였다.

print(img['pt'])
img

SC이슈]"컴백 NO, 예능은 OK"…우주소녀 성소, 中스케줄 논란 ...


{'cb': 12,
 'cl': 15,
 'cr': 21,
 'id': 'Xm3IhVOeFnfSAM:',
 'isu': 'news.chosun.com',
 'itg': 0,
 'ity': 'jpg',
 'oh': 782,
 'ou': 'http://image.chosun.com/sitedata/image/201809/03/2018090301334_0.jpg',
 'ow': 540,
 'pt': 'SC이슈]"컴백 NO, 예능은 OK"…우주소녀 성소, 中스케줄 논란 ...',
 'rh': 'news.chosun.com',
 'rid': 'eNkCp7MbYFuoPM',
 'rmt': 0,
 'rt': 0,
 'ru': 'http://news.chosun.com/site/data/html_dir/2018/09/03/2018090301347.html',
 'sc': 1,
 'st': '조선일보',
 'th': 270,
 'tu': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWPUERnmXHTdJA81s9gdcNo5tk6X9XEshi_O7ukJdoLWZcHMCtxw',
 'tw': 186}

---

# 네이버 블로그에서 실습
- 네이버 블로그
- Focused Crawling - Depth : 어제 배웠던 내용 활용
- image scarping : 오늘 배운 내용 활용

In [ ]:
# 실습용 폴더 생성

import os 

os.mkdir("./6일차_실습_image")
os.mkdir("./6일차_실습_html")

In [340]:
naverUrl = "https://search.naver.com/search.naver"
params = "sm=top_hty&fbm=1&ie=utf8&query=성소"
params = params.split("&")
params = {_.split("=")[0] : _.split("=")[1]  for _ in params}

resp = download("get", naverUrl, params=params, headers=headers)
dom = BeautifulSoup(resp.text, "lxml")

naverResults = [(_["href"], 1) for _ in dom.select(".section .type01 dt > a")]

In [127]:
resp.headers['Content-Type']

'text/html;charset=UTF-8'

In [140]:
# 어제 만든 코드를 그대로 이용했다.

URLs = list()
# 이 url로 request를 보내면 바로 error가 난다.
# 네이버 블로그에 이 url이 링크로 심어져있다. 
# visitedURLs에 미리 넣어두고 걸러낼거다.
visitedURLs = [('http://security.naver.com/index.nhn', 1)] 
URLs.extend(naverResults)

lastURL = ""

# baseURL = "blog.naver.com"
baseURL = ["blog.naver.com", "post.naver.com", "m.naver.com"]
useBase = False

useDepth = 3

while URLs:
    urls = URLs.pop(-1)
    
    url = urls[0]
    depth = urls[1] + 1 
    print(url)
    
    visitedURLs.append(urls)

    resp = download("get", url, headers=headers)
    
    if not resp:
        if 500 <= resp.status_code < 600:
            URLs.append(url)
        continue
    
    # 이미지 스크래핑
    if resp.headers["Content-Type"].split("/")[0] == "image":
        imgTitle = resp.url.split("/")[-1].replace("?", "")
        imgExt = ext[resp.headers["Content-Type"].split("/")[-1]]
        with open("6일차_실습_image/"+imgTitle+"."+imgExt, "wb") as fp:
            fp.write(resp.content)
        continue
    else:
        htmlTitle = resp.url.split("/")[-1].replace("?","")
        with open("6일차_실습_html/"+htmlTitle+".html", "w", encoding="utf-8") as fp:
            fp.write(resp.text)
        
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    # Iframe 처리
    for _ in [requests.compat.urljoin(url, _["src"])
             for _ in dom.select("iframe") 
              if _.has_attr("src") and 
                _.has_attr("id") and
                _["id"] == "mainFrame"]:
        iframe = download("get", _, headers=headers)
        iframeDom = BeautifulSoup(iframe.text, "lxml")
    
        links = [requests.compat.urljoin(url, _["href"])  
                 for _ in iframeDom.select("a")
                 if _.has_attr("href") and 
                 requests.utils.urlparse(_["href"])[2]]
        
        # 페이지 내에 image가 있으면 이미지 주소를 crawling 하는 부분.
        links.extend([requests.compat.urljoin(url, _["src"])
                     for _ in iframeDom.select("img")
                     if _.has_attr("src")])

        for link in links:
            if useBase and requests.utils.urlparse(link)[1] in baseURL:     
                if link not in [_[0] for _ in URLs] and \
                    link not in [_[0] for _ in visitedURLs]:
                    URLs.append((link, depth))
            if depth < useDepth:
                if link not in [_[0] for _ in URLs] and \
                    link not in [_[0] for _ in visitedURLs]:
                    URLs.append((link, depth))
    
    # Link 처리
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]
        
    for link in links:
        if useBase and requests.utils.urlparse(link)[1] in baseURL:     
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
        if depth < useDepth:
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
    
    print(len(URLs), len(visitedURLs))
    
#     if requests.utils.urlparse(url)[1]  == lastURL:
#         time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

https://blog.naver.com/cpflqmffor?Redirect=Log&logNo=221604798149
80 2
https://blogpfthumb-phinf.pstatic.net/MjAxODExMTVfMTIx/MDAxNTQyMjA5ODUzNTc2.T3WTBwbC65vaDA-B39Kqy5RGcMHi2952KQJaRMJ_KA8g.qqF7ItgPnlTcl0D06IOv7ko4eRjo4iDDd86xS68Uwwgg.JPEG.cpflqmffor/20171112105032_IMG_0629.JPG?type=s40
https://blogimgs.pstatic.net/static/ws/btn_close.gif
https://blogimgs.pstatic.net/imgs/btn_cancel_pop2.gif
https://blogimgs.pstatic.net/imgs/btn_confirm_pop2.gif
https://blogimgs.pstatic.net/nblog/guestbook/btn_close2.gif
https://blogimgs.pstatic.net/nblog/guestbook/btn_ok.gif
https://blogimgs.pstatic.net/nblog/widget/btn_close.gif
https://blogimgs.pstatic.net/static/common/popup/btn_close3.gif
https://blogimgs.pstatic.net/nblog/book/publishingcompany/btn_close.gif
https://blogimgs.pstatic.net/nblog/ico_notice2.gif
https://blogimgs.pstatic.net/nblog/mylog/post/tit_viewexif.gif
https://blogimgs.pstatic.net/nblog/mylog/post/btn_originaldn.gif
https://blogimgs.pstatic.net/nblog/mylog/post/btn_viewexif.gi

17 81
https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/noemisuh/221628595728
16 82
https://nid.naver.com/nidlogin.login?mode=form&url=https://blog.naver.com/noemisuh
15 83
https://blog.naver.com/PostList.nhn?blogId=noemisuh
14 84
https://ponciano.blog.me/221557473384
13 85
https://blog.aladin.co.kr/fallen77/10379774
158 86
https://blog.aladin.co.kr/fallen77/subscript
157 87
http://blog.aladin.co.kr/fallen77/10596838 
156 88
http://blog.aladin.co.kr/fallen77/10712191 
155 89
http://blog.aladin.co.kr/fallen77/10730613 
154 90
http://blog.aladin.co.kr/fallen77/10777274 
153 91
http://blog.aladin.co.kr/fallen77/10804923 
152 92
http://blog.aladin.co.kr/fallen77/10826916 
151 93
https://blog.aladin.co.kr/fallen77/10842140 
150 94
https://blog.aladin.co.kr/fallen77/10879296 
149 95
https://blog.aladin.co.kr/fallen77/10947079 
148 96
https://blog.aladin.co.kr/fallen77/11005363 
147 97
https://blog.aladin.co.kr/fallen77/11071125#C2921172
146 98
https://blog.aladin.co.k

InvalidSchema: No connection adapters were found for 'javascript:moveBlogDateURL()'

In [136]:
resp.url

'https://blog.naver.com/FILEPATH'

In [141]:
url

'javascript:moveBlogDateURL()'

---

# 뽐뿌에서 실습
#### 미션: 게시판에서 제목, 추천수, link를 가져온다.

- https://ppomppu.co.kr
- 국내 대형 커뮤니티 중 하나이다. (그럼에도 불구하고 거지같이 만들었다.)
- 전형적인 한국형 웹사이트이다. (거지같이 만들었다.)
- Not Well-foremd HTML이다. (거지같다는 뜻이다.)

- 웹사이트를 잘 만들었는지 검사해주는 사이트
- https://validator.w3.org/
    - well-formed HTML인지 검사한다.
    - HTML 태그가 제대로 닫혀있는지 등을 검사해준다.
    - ppomppu가 Not Well-formed 웹사이트라는 것을 알 수 있다.

In [142]:
resp = download("get", "https://www.ppomppu.co.kr/zboard/zboard.php",
               headers=headers, params={"id":"ppomppu"})

# Not Well-formed이면 "lxml" 파서보다 "html.parser" 파서가 더 좋다.
dom = BeautifulSoup(resp.text, "html.parser")

In [144]:
# encoding이 euc-kr이다.  =>  비표준임.
# encoding이 utf-8이 표준이 되었음.
# 현재 쓰는 requests 패키지는 내부적으로 encoding을 처리해서 에러가 안나지만, 
# 하위 패키지를 사용하면 encoding 옵션을 안 바꿔주면 에러가 날 것이다. ex) urllib

resp.encoding

'euc-kr'

In [148]:
# class^=~ 는 "~로 시작하는 클래스" 라는 뜻이다.

len([_ for _ in dom.select("tr[class^=list]")
    if not _.has_attr("id")]), \
len(dom.select("tr.list0, tr.list1"))

(20, 20)

In [149]:
tr = [_ for _ in dom.select("tr[class^=list]")
     if not _.has_attr("id")]

In [161]:
items = list()

for _ in tr:
    td = _.find_all(recursive=False)
    title = td[3].select_one("a > font").text
    url = requests.compat.urljoin(resp.url, 
                                 td[3].select_one("a > font").find_parent()["href"])
    # 추천수가 없을 수도 있으므로, 에러 처리한다.
    hits = 0 if len(td[5].text) < 1 \
                   else td[5].text.split("-")[0].strip()
    
    resp = download("get", url)
    itemDom = BeautifulSoup(resp.text, "lxml")
    images = [requests.compat.urljoin(url, _["src"]) 
             for _ in itemDom.select("td.board-contents img")
             if _.has_attr("src")]

    items.append({"title":title, "url":url, "hits":hits, "img":images})

#     print(td[3].text.strip())
#     print(td[5].text)
#     print(td[3].select_one("a > font").find_parent()["href"])
#     print(td[3].select_one("a > font").text)
#     print(td[5].text.split("-"))
#     break

In [162]:
items

[{'title': '[위메프]메디어스 마스크팩 30매 균일가(4900원/무료)',
  'url': 'https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=1&divpage=57&no=328836',
  'hits': '1',
  'img': ['https://cdn.ppomppu.co.kr/zboard/data3/2019/0910/20190910154718_niqbtgmu.png',
   'https://cdn.ppomppu.co.kr/zboard/data3/2019/0910/20190910154721_xzepsqix.png']},
 {'title': '[인터파크] 콜맨 아웃도어 웨건  (신한카드 96,800/배송비무료)',
  'url': 'https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=1&divpage=57&no=328834',
  'hits': '2',
  'img': ['https://cdn.ppomppu.co.kr/zboard/data3/2019/0910/20190910143304_qwpzzqtl.jpeg',
   'https://cdn.ppomppu.co.kr/zboard/data3/2019/0910/20190910144150_dyhhynfn.jpeg',
   'https://cdn.ppomppu.co.kr/zboard/data3/2019/0910/20190910143353_plwonfwu.jpeg']},
 {'title': '[위메프] 마이크로소프트 Sculpt Ergonomic Mouse 무선 마우스 병행&벌크 (36,000/2,500)',
  'url': 'https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=1&divpage=57&no=328833',
  'hits': 0,
  'img': ['https://cdn.ppomppu.co.kr/zboard/data3/2019/09

### 추가 미션

In [165]:
# title에서 가격만 가져오기
import re
re.findall(r"\(\s*([0-9,.]+)", items[0]["title"])

['4900']

In [172]:
for _ in items:
    print(re.findall(r"\([^0-9]*([0-9,.]+)", _["title"])[-1].replace(",", ""))
#     print(_["title"])

4900
96800
36000
1499000
11990
12930
799000
100
16900
2500
22900
149000
1370837
8500
51400
21900
2400
3
597650
649000


---



# 과제: 뽐뿌 자유게시판
- 제목, 링크, 댓글 가져오기.

### 과제 최종
- 중간 과정은 아래에 있음.

In [299]:
url ="http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard"
resp = download("get", url, headers=headers)
dom = BeautifulSoup(resp.text, "html.parser")
tr = [_ for _ in dom.select("tr[class^=list]") if not _["class"] == ["list_notice"]]

results = []

for _ in tr:
    title = _.select("td")[2].select_one("a").text
    link = requests.compat.urljoin(url, 
                                  _.select("td")[2].select_one("a")["href"])
    # 댓글 있는지 확인하고 가져오기
    if not tr[0].select("td")[2].select(".list_comment2"):
        linkResp = download("get", link, headers=headers)
        linkDom = BeautifulSoup(linkResp.text, "html.parser")
        comments = [(_.select_one("b a").text, _.select_one(".han").text) for _ in linkDom.select(".comment_wrapper")]
    else:
        comments = []
        
    result = {"title":title, "link":link, "comments":comments}
    results.append(result)

In [300]:
results

[{'title': '주거지 열람 제한 쉽네요.',
  'link': 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1236&no=6624867',
  'comments': []},
 {'title': '일본 보이그룹이 한국 등 아시아에서 먹히지 않는 이유???',
  'link': 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1236&no=6624866',
  'comments': [('조가라그내', '일본에도 잘생긴 애들 많던데 쟤들은 뭔지'),
   ('뽐뿌가쟈', '무슨 만화인줄.... 오버하지마'),
   ('까칠단혼', '왜저러는 걸까요...'),
   ('', '합격보다 웃음에 중점을 둔 출연 아닐까....하네요'),
   ('sekai', '밑엔 그니마 봐줄만 하네요\n몇살이길래\n\xa0'),
   ('우리결국했어요', '내 손발...'),
   ('더블핫팩', '문복이 가면 탑 먹겠는데요 ㅎ\xa0'),
   ('파란돌맹', 'ㅋㅋ 코미디 프로그램인가보네요'),
   ('', '쟤넨\xa0 남아이돌은 호스트 같음'),
   ('', '아..주먹이 운다..')]},
 {'title': '와.. 닭발 기사 이제 봤는데 진짜 충격적이네요.',
  'link': 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1236&no=6624865',
  'comments': []},
 {'title': '뇌파에서 기억을 뽑을수만 있다면',
  'link': 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1236&no=6624864',
  'comments': [('', '나는 썩었어..'),
   ('', '인간은 망각의 동물이기때문에

> 아래는 위 과제 결과에 이르기 까지의 과정입니다.

In [291]:
url ="http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard"
resp = download("get", url, headers=headers)
dom = BeautifulSoup(resp.text, "html.parser")

In [301]:
# 공지사항을 제외하고 게시판 목록을 가져온다.

tr = [_ for _ in dom.select("tr[class^=list]") if not _["class"] == ["list_notice"]]
len(tr)

30

In [302]:
title = tr[15].select("td")[2].select_one("a").text
title

'배민에서  주로  뭐시켜드세요?'

In [303]:
# 댓글이 있는지 없는지 확인 => list_comment2가 없으면 댓글 없음.
# 제목 옆에 댓글 갯수임.

tr[0].select("td")[2].select(".list_comment2")

[]

In [304]:
# 댓글이 있으면 span의 text를 찍으면 된다.

tr[15].select("td")[2].select_one(".list_comment2 > span").text

'15'

In [305]:
# link는 urljoin까지 해야 완성형 url이 된다.

link = tr[15].select("td")[2].select_one("a")["href"]
link = requests.compat.urljoin(url, link)
link

'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1236&no=6624852'

In [306]:
# 댓글을 얻기 위해서 link를 들어간다.

linkResp = download("get", link, headers=headers)
linkDom = BeautifulSoup(linkResp.text, "html.parser")

In [331]:
for _ in linkDom.select(".comment_wrapper"):
    print(_.name)
    print(_.select("b a"))

div
[<a href="#" onclick="return false">노돌</a>]
div
[<a href="#" onclick="return false">수마수</a>]
div
[<a href="#" onclick="return false">오르테가</a>]
div
[<a href="#" onclick="return false"><img align="absmiddle" alt="까칠한쉘든" border="0" src="//nic.ppomppu.co.kr/zboard/nickcon/63223.gif?v=201909101749" vspace="5"/></a>]
div
[<a href="#" onclick="return false">부산강냉이</a>]
div
[<a href="#" onclick="return false">happysmile</a>]
div
[<a href="#" onclick="return false">만만87</a>]
div
[<a href="#" onclick="return false">오늘하늘맑음</a>]
div
[<a href="#" onclick="return false">조가라그내</a>]
div
[<a href="#" onclick="return false">우체국</a>]
div
[<a href="#" onclick="return false">바닐라라떼죠아</a>]
div
[<a href="#" onclick="return false">바닐라라떼죠아</a>]
div
[<a href="#" onclick="return false">바닐라라떼죠아</a>]
div
[<a href="#" onclick="return false">바닐라라떼죠아</a>]
